In [1]:
import datetime
import numpy as np
import pandas as pd
from sklearn.externals import joblib

CT_DIR = 'save/'
import pickle
def load_obj(name):
    with open(CT_DIR + name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [2]:
# Liste des companies
model_carrier = load_obj('model_carrier')
choices_carrier = [(x, x) for x in model_carrier]
# Liste des aéroports
model_airport = load_obj('model_airport')
choices_airport = model_airport.get_values()
choices_airport

array([[10141, 'Aberdeen, SD'],
       [10136, 'Abilene, TX'],
       [10165, 'Adak Island, AK'],
       [10732, 'Aguadilla, PR'],
       [10874, 'Akron, OH'],
       [10146, 'Albany, GA'],
       [10257, 'Albany, NY'],
       [10140, 'Albuquerque, NM'],
       [10185, 'Alexandria, LA'],
       [10135, 'Allentown/Bethlehem/Easton, PA'],
       [10333, 'Alpena, MI'],
       [10279, 'Amarillo, TX'],
       [10299, 'Anchorage, AK'],
       [10408, 'Appleton, WI'],
       [10157, 'Arcata/Eureka, CA'],
       [10431, 'Asheville, NC'],
       [10372, 'Aspen, CO'],
       [10397, 'Atlanta, GA'],
       [10158, 'Atlantic City, NJ'],
       [10208, 'Augusta, GA'],
       [10423, 'Austin, TX'],
       [10561, 'Bakersfield, CA'],
       [10821, 'Baltimore, MD'],
       [10581, 'Bangor, ME'],
       [10754, 'Barrow, AK'],
       [10781, 'Baton Rouge, LA'],
       [10728, 'Beaumont/Port Arthur, TX'],
       [10666, 'Bellingham, WA'],
       [10631, 'Bemidji, MN'],
       [14489, 'Bend/Redmond, OR']

In [3]:
CT_COMPANY = "company"
CT_DEPARTURE = "departure"
CT_DATE = "date"
CT_TIME = "time"

# Liste des jours fériés américains en  2016
holidays = ['2016-01-01', '2016-01-18', '2016-02-15', '2016-05-30', '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-26']
dfhol = pd.DataFrame({'date' : holidays}, columns = ['date'])
dfhol = pd.to_datetime(dfhol.date)

def DaysToHoliday(dt):
    if dt.year != 2016:
        # On renvoie une valeur moyenne si on n'est pas en 2016
        return 10
    dd = datetime.datetime(2016, dt.month, dt.day)
    return (dfhol - dd).abs().min().days


In [4]:
myCOMPANY = 'AA'
myDEPARTURE = 10397 #Atlanta
myDATE = datetime.datetime.strptime('2018-02-07', "%Y-%m-%d")
myTIME = '00:00'

In [6]:
model_name = 'model_columns_' + myCOMPANY
model_df = load_obj(model_name)
tbl = [0 for _ in range(len(model_df.columns))]
if len(tbl) == 0:
    print(model_name + ' not found')


In [7]:
hDay = DaysToHoliday(myDATE)
print(hDay)

10


In [8]:
model_df.loc[0] = 0

In [9]:
model_df["HDAYS"] = hDay
model_df["MONTH_" + str(myDATE.month)] = 1
model_df["DAY_OF_WEEK_" + str(myDATE.weekday())] = 1
hh, mm = [int(x) for x in myTIME.split(":")]
model_df["DEP_HOUR_" + str(hh)] = 1
col = "ORIGIN_AIRPORT_ID_" + str(myDEPARTURE)
if col not in model_df.columns:
    print("No departure from " + str(myDEPARTURE) + " for company" + myCOMPANY)

model_df["ORIGIN_AIRPORT_ID_" + str(myDEPARTURE)] = 1
model_df

,HDAYS,MONTH_1,MONTH_10,MONTH_11,MONTH_12,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,...,DEP_HOUR_7,DEP_HOUR_8,DEP_HOUR_9,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6,DAY_OF_WEEK_7
0,10,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [10]:
scaler = joblib.load(CT_DIR + 'model_scaler_' + myCOMPANY + '.pkl')
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [11]:
xnum = 1
x_numerical = model_df.iloc[:, 0:xnum]
x_numerical = scaler.transform(x_numerical )
model_df.loc[:, 0:xnum] = x_numerical


In [12]:
model = joblib.load(CT_DIR + 'model_SGD_' + myCOMPANY + '.pkl')
y_pred = model.predict(model_df)[0]
y_pred

-10.662383113329387